In [7]:
from random import random
import numpy as np
import time

In [8]:
def tsp_read(nodes):
    infile = open(nodes, 'r')
    content = infile.readline().strip().split()
    print("File Name: ", content[2])

    while content[0] != 'NODE_COORD_SECTION':
        if(content[0] == 'DIMENSION'):
            dimension = content[2]
        content = infile.readline().strip().split()
    nodelist = []
    placelist = []
    print('Dimension', dimension)
    N = int(dimension)
    for i in range(0, N):
        x, y, z = infile.readline().strip().split()[:]
        nodelist.append([float(y), float(z)])
        placelist.append(x)
    
    infile.close()
    return nodelist, placelist

In [ ]:
def euclidean_distance(nodes, n1, n2):
    distance = sqrt((nodes[n1][0]-nodes[n2][0])**2 + (nodes[n1][1]-nodes[n2][1])**2)
    return distance

In [ ]:
def cooling_function(T):
    return T * 0.9

In [ ]:
def simulated_annealing(nodes):
    unvisited = nodes
    initial = random.randint(0, len(nodes)-1)
    pathlist = []
    pathlist.append(nodes[initial])
    unvisited.pop(initial)
    T = 50
    bestnode = initial
    while len(unvisited) != 0:
        idx = random.randint(0, len(unvisited)-1)
        neighbour = unvisited[idx]
        delE = euclidean_distance(nodes, initial, idx)
        prob = 1 / (1 + exp(delE / T))
        if random.random() < prob:
            
        

In [9]:
def main():
    # generate_random_coords(100)
    nodes, place = tsp_read("Data/Rajasthan.tsp")
    coords = np.array(nodes)
    n = len(coords)
    start = time.time_ns()
#     sa = SimAnneal(coords, place, stopping_iter=n*10000000)
    end = time.time_ns()
#     print('Execution Time', end-start)
#     sa.simulated_annealing()
#     sa.display_optimal_path()
#     sa.animateSolutions()
#     sa.plot_learning()


if __name__ == "__main__":
    main()

File Name:  Rajasthan
Dimension 20
